In [2]:
import pandas as pd
import importlib
import process
importlib.reload(process)
from process import *
import os

In [3]:
import dgl
import torch
import os
from dgl.data.utils import load_graphs


In [4]:
df_or = pd.read_csv("/home/niamatzawad/niamatzawad/Datasets/ACLED/1997-01-01-2024-11-03.csv") 


In [5]:
device = torch.device('cuda:0')
time_window = 3

In [6]:
df, mapping_entity_note_dict, mapping_entity_ts_dict, mapping_entity_int_dict, mapping_entity_se_dict = preprecoss_acled(df_or.copy())

In [125]:
len(mapping_entity_int_dict.keys())

18622

In [7]:
df['actor1_i'].iloc[50]


2

In [8]:
df.sub_event_type.unique()

array(['Armed clash', 'Attack', 'Looting/property destruction',
       'Government regains territory',
       'Excessive force against protesters',
       'Remote explosive/landmine/IED', 'Protest with intervention',
       'Abduction/forced disappearance', 'Violent demonstration',
       'Non-state actor overtakes territory', 'Mob violence',
       'Shelling/artillery/missile attack', 'Disrupted weapons use',
       'Grenade', 'Non-violent transfer of territory', 'Air/drone strike',
       'Sexual violence', 'Arrests', 'Other', 'Change to group/activity',
       'Agreement', 'Suicide bomb', 'Chemical weapon', 'Peaceful protest',
       'Headquarters or base established'], dtype=object)

In [11]:
relation_types = []
for se in df.sub_event_type.unique():
    relation_types.append('')

In [12]:
# Group by 'year' and split into multiple DataFrames
grouped_dfs = {year: group for year, group in df.copy().groupby('year')}

# grouped_dfs = {}
# for year, group in df.copy().groupby('year'):
#     if year > 2016:
#         grouped_dfs[year] = group
# Access individual DataFrames
# for year, df_year in grouped_dfs.items():
#     print(f"DataFrame for year {year}:\n", df_year, "\n")


In [55]:
grouped_dfs[1997].columns

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'civilian_targeting', 'iso', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'tags', 'timestamp', 'notes_i', 'ts', 'actor1_i', 'actor2_i',
       'sub_event_type_i', 'actor1_counts', 'actor2_counts',
       'sub_event_type_counts', 'actor1_inverse_weight',
       'actor2_inverse_weight', 'sub_event_type_inverse_weight',
       'edge_inverse_weight', 'normalized_actor1_inverse_weight',
       'normalized_actor2_inverse_weight', 'normalized_edge_inverse_weight',
       'normalized_sub_event_type_inverse_weight'],
      dtype='object')

In [36]:
grouped_dfs.keys()

dict_keys([1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024])

In [13]:
import random
graphs = []

for df_dict in grouped_dfs.values():
    total_actors = len(mapping_entity_int_dict.keys())
    total_actions = len(mapping_entity_se_dict.keys())

    node_data = {
        "actor": total_actors,
        "action": total_actions
    }


    # edge_data = {
    #     ("actor", "involved_in", "action"): (torch.randint(0, len(pd.concat([df_dict['actor1_i'], df_dict['actor2_i']]).unique()), (1000,)), torch.randint(0, len(df_dict['sub_event_type'].unique()), (1000,)))
    # }

    
    src_nodes = torch.arange(0, total_actors//2, 1) 
    dst_nodes = torch.randint(0, total_actions, (total_actors//2,))  # Actress indices

    edge_data = {("actor", "involved_in", "action"): (src_nodes, dst_nodes)}

    graph = dgl.heterograph(edge_data, num_nodes_dict=node_data)


    # graph.nodes["action"].data["features"] = torch.randn(node_data["action"], 128)
    graph.nodes["action"].data["_ID"] = torch.arange(0, total_actions)
    
    # graph.nodes["actor"].data["feat"] = torch.randn(node_data["actor"], 128)
    graph.nodes["actor"].data["_ID"] = torch.arange(0, total_actors)

    

    graphs.append(graph)

In [14]:
graphs[0]

Graph(num_nodes={'action': 25, 'actor': 18622},
      num_edges={('actor', 'involved_in', 'action'): 9311},
      metagraph=[('actor', 'action', 'involved_in')])

In [47]:
df

,col1,col2
0,0,2
1,1,3
2,2,0
3,3,0


In [44]:
import pandas as pd
from collections import defaultdict


def get_edges(df_dict, col_1, col_2):
    # Initialize a defaultdict
    result = defaultdict(set)

    # Populate the dictionary with connections
    for col1, col2 in zip(df_dict[col_1], df_dict[col_2]):
        result[col1].add(col2)  # Add the connection from col1 to col2
        result[col2].add(col1)  # Add the reverse connection from col2 to col1

    # Convert sets to lists and the defaultdict to a dict
    result = {key: list(value) for key, value in result.items()}

    return result

def get_edges_unidirectional(df_dict, col_1, col_2):
    # Initialize a defaultdict
    result = defaultdict(set)

    # Populate the dictionary with connections
    for col1, col2 in zip(df_dict[col_1], df_dict[col_2]):
        result[col1].add(col2)  # Add the connection from col1 to col2

    # Convert sets to lists and the defaultdict to a dict
    result = {key: list(value) for key, value in result.items()}

    return result

def get_edges_action(df_dict,action):
    # Initialize a defaultdict
    result = defaultdict(set)

    temp_df = df_dict[df_dict['sub_event_type'] == action]
    # Populate the dictionary with connections
    for col1, col2 in zip(temp_df['actor1_i'], temp_df['sub_event_type_i']):
        result[col1].add(col2)  # Add the connection from col1 to col2
        result[col2].add(col1)  # Add the reverse connection from col2 to col1

    # Convert sets to lists and the defaultdict to a dict
    result = {key: list(value) for key, value in result.items()}

    return result

In [46]:
get_edges_unidirectional(df, 'actor1_i', 'sub_event_type_i')

{0: [0, 14],
 1: [0, 1, 2, 3, 7, 9, 11, 12, 13, 14, 15, 16, 17, 18, 20],
 2: [0, 1, 3, 5, 11],
 3: [0, 1, 2, 3, 7, 9, 13, 16, 20],
 4: [0, 1, 2, 3, 9],
 5: [0, 1, 2, 5, 7, 11, 13, 16],
 6: [0, 1, 3, 5, 9, 11, 14, 15],
 7: [0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20],
 8: [0, 1, 2, 3, 7, 9],
 9: [0, 1, 3, 4, 5, 6, 7, 8, 9, 11, 15, 16, 17],
 10: [0, 1, 3, 4, 6, 8, 10, 12, 17],
 11: [4, 6, 23],
 12: [0, 1, 3, 7, 9, 16, 20],
 13: [0, 1, 3, 6, 9, 11, 15],
 14: [5],
 15: [0, 1, 2, 3, 5, 9, 11, 13, 15, 16, 20, 22],
 16: [0, 1, 2, 5, 7, 11, 13, 18, 21],
 17: [0, 1, 2, 3, 9, 11],
 18: [0, 1],
 19: [0],
 20: [4, 6],
 21: [0, 1, 7],
 22: [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21],
 23: [0, 1],
 24: [0, 1, 2, 3, 5, 7, 9, 10, 11, 13, 15, 16, 17, 18, 19, 20, 24],
 25: [0],
 26: [0, 1, 17, 7],
 27: [0, 1, 2, 5, 7, 11, 13, 16, 19, 20],
 28: [0, 1, 2, 3, 5, 7, 9, 11, 13, 14, 15, 16, 17, 18, 19, 20],
 29: [8, 10],
 30: [0, 1, 3, 9],
 31: [0, 1, 2, 3, 5, 7

In [34]:
def get_src_dst_tensors(d):
    src = []
    dst = []
    
    for key, values in d.items():
        src.extend([key] * len(values))  # Add the key repeated for each value
        dst.extend(values)              # Add all connected values
    src_tensor = torch.tensor(src)
    dst_tensor = torch.tensor(dst)
    # print("Source Tensor:", src_tensor)
    # print("Destination Tensor:", dst_tensor)

    return src,dst

In [35]:
get_src_dst_tensors(get_edges(grouped_dfs[1997], 'actor1_i', 'sub_event_type_i'))

{0: [0, 1, 2, 3, 4, 6, 7, 8, 10, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 33, 34, 35, 39, 43, 45, 47, 48, 56, 57, 58, 59, 61, 64, 66, 68, 70, 72, 73, 74, 75, 76, 77, 80, 82, 83, 88, 91, 92, 94, 95, 99, 101, 103, 104, 105, 107, 109, 110, 112, 113, 114, 119, 120, 121, 123, 125, 126, 127, 130, 134, 137, 138, 143, 147, 148, 149, 150, 152, 157, 160, 161, 165, 167, 169, 170, 171, 173, 175, 176, 178, 179, 181, 182, 183, 185, 186, 188, 189, 191, 193, 195, 199, 200, 202, 203, 204, 208, 209, 211, 212, 213, 217, 218, 219, 223, 226, 227, 229, 230], 1: [0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 13, 15, 16, 17, 18, 21, 22, 23, 24, 26, 27, 28, 30, 31, 33, 35, 37, 40, 42, 44, 47, 54, 55, 56, 59, 60, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 80, 82, 83, 86, 91, 95, 96, 101, 104, 105, 106, 115, 119, 120, 125, 127, 131, 132, 134, 136, 141, 142, 146, 149, 150, 151, 152, 153, 154, 159, 160, 162, 163, 165, 166, 171, 172, 174, 175, 176, 180, 184, 187, 190, 192, 196, 197, 198, 200, 2

([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [40]:
# dgl.save_graphs("graph_data_acled.bin", graphs)

In [103]:
df

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,actor2_counts,sub_event_type_counts,actor1_inverse_weight,actor2_inverse_weight,sub_event_type_inverse_weight,edge_inverse_weight,normalized_actor1_inverse_weight,normalized_actor2_inverse_weight,normalized_edge_inverse_weight,normalized_sub_event_type_inverse_weight
2307613,CEN1,1997-01-01,1997,1,Political violence,Battles,Armed clash,Military Forces of the Central African Republi...,Mutiny of Military Forces of the Central Afric...,State forces,...,4,365035,0.333333,0.250000,0.000003,0.583333,0.333319,0.249983,0.291651,0.000000
2307596,RWA3,1997-01-01,1997,1,Political violence,Battles,Armed clash,Military Forces of Rwanda (1994-),NaN,State forces,...,560,365035,0.002584,0.001786,0.000003,0.004370,0.002563,0.001763,0.002163,0.000000
2307597,ALG1,1997-01-01,1997,1,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,NaN,Rebel group,...,838,225068,0.002203,0.001193,0.000004,0.003396,0.002182,0.001171,0.001676,0.000019
2307598,SIE1,1997-01-01,1997,3,Strategic developments,Strategic developments,Looting/property destruction,RUF: Revolutionary United Front,NaN,Rebel group,...,2161,44517,0.000883,0.000463,0.000022,0.001345,0.000862,0.000440,0.000651,0.000217
2307599,SIE4,1997-01-01,1997,3,Strategic developments,Strategic developments,Looting/property destruction,Kamajor Militia,NaN,Political militia,...,2161,44517,0.009346,0.000463,0.000022,0.009809,0.009325,0.000440,0.004883,0.000217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734,RUS25496,2024-10-25,2024,1,Political violence,Battles,Armed clash,Military Forces of Russia (2000-),NaN,State forces,...,44482,365035,0.000021,0.000022,0.000003,0.000043,0.000000,0.000000,0.000000,0.000000
735,RUS25497,2024-10-25,2024,1,Strategic developments,Strategic developments,Disrupted weapons use,Military Forces of Russia (2000-),NaN,State forces,...,6174,22160,0.000021,0.000162,0.000045,0.000183,0.000000,0.000139,0.000070,0.000466
738,RUS25500,2024-10-25,2024,1,Political violence,Explosions/Remote violence,Air/drone strike,Military Forces of Russia (2000-) Air Force,Military Forces of Russia (2000-),State forces,...,44482,71439,0.000164,0.000022,0.000014,0.000187,0.000143,0.000000,0.000072,0.000124
720,PAK146247,2024-10-25,2024,1,Political violence,Battles,Armed clash,TTP: Tehreek-i-Taliban Pakistan,NaN,Rebel group,...,1118,365035,0.000358,0.000894,0.000003,0.001252,0.000337,0.000872,0.000605,0.000000


In [ ]:
def evaluate(model, val_feats, val_labels):
    val_mae_list, val_rmse_list = [], []
    model.eval()
    with torch.no_grad():
        for (G_feat, (pos_label, neg_label)) in zip(val_feats, val_labels):

            G_feat = G_feat.to(device)
            pos_label = pos_label.to(device)
            neg_label = neg_label.to(device)

            h = model[0](G_feat, 'author')
            pos_score = model[1](pos_label, h)
            neg_score = model[1](neg_label, h)

            loss = compute_loss(pos_score, neg_score, device)
            auc, ap = compute_metric(pos_score, neg_score)
    
    return auc, ap

In [36]:
subevent_dict = df['sub_event_type'].value_counts().to_dict()

In [37]:
k_subevent, v_subevent = [],[]
for key,value in subevent_dict.items():
    k_subevent.append(key)
    v_subevent.append(value)

edges = {}
for e in k_subevent:
    edges[("actor","involved_in_action", e)] = get_src_dst_tensors(get_edges_action(grouped_dfs[1997], e))

In [38]:
edges

{('actor',
  'involved_in_action',
  'Armed clash'): ([0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   6,
   7,
   4,
   10,
   12,
   17,
   19,
   23,
   13,
   25,
   26,
   31,
   34,
   15,
   2,
   39,
   27,
   43,
   45,
   8,
   47,
   48,
   57,
   58,
   61,
   70,
   74,
  

In [39]:
subevent_dict

{'Armed clash': 365035,
 'Attack': 225068,
 'Shelling/artillery/missile attack': 77961,
 'Mob violence': 74982,
 'Air/drone strike': 71439,
 'Remote explosive/landmine/IED': 47892,
 'Violent demonstration': 44575,
 'Looting/property destruction': 44517,
 'Protest with intervention': 41132,
 'Abduction/forced disappearance': 28945,
 'Disrupted weapons use': 22160,
 'Arrests': 20386,
 'Other': 12031,
 'Government regains territory': 10848,
 'Non-state actor overtakes territory': 7178,
 'Peaceful protest': 7055,
 'Grenade': 6440,
 'Excessive force against protesters': 5637,
 'Sexual violence': 4578,
 'Agreement': 3036,
 'Non-violent transfer of territory': 2232,
 'Suicide bomb': 2124,
 'Change to group/activity': 1089,
 'Chemical weapon': 99,
 'Headquarters or base established': 11}

In [ ]:
df.columns

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'civilian_targeting', 'iso', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'tags', 'timestamp', 'notes_i', 'ts', 'actor1_i', 'actor2_i',
       'sub_event_type_i', 'actor1_counts', 'actor2_counts',
       'sub_event_type_counts', 'actor1_inverse_weight',
       'actor2_inverse_weight', 'sub_event_type_inverse_weight',
       'edge_inverse_weight', 'normalized_actor1_inverse_weight',
       'normalized_actor2_inverse_weight', 'normalized_edge_inverse_weight',
       'normalized_sub_event_type_inverse_weight'],
      dtype='object')

In [112]:
x = {}
x[('a','b')] = (1,2)
x

{('a', 'b'): (1, 2)}

{'Armed clash': 0,
 'Attack': 1,
 'Looting/property destruction': 2,
 'Government regains territory': 3,
 'Excessive force against protesters': 4,
 'Remote explosive/landmine/IED': 5,
 'Protest with intervention': 6,
 'Abduction/forced disappearance': 7,
 'Violent demonstration': 8,
 'Non-state actor overtakes territory': 9,
 'Mob violence': 10,
 'Shelling/artillery/missile attack': 11,
 'Disrupted weapons use': 12,
 'Grenade': 13,
 'Non-violent transfer of territory': 14,
 'Air/drone strike': 15,
 'Sexual violence': 16,
 'Arrests': 17,
 'Other': 18,
 'Change to group/activity': 19,
 'Agreement': 20,
 'Suicide bomb': 21,
 'Chemical weapon': 22,
 'Peaceful protest': 23,
 'Headquarters or base established': 24}

In [55]:
import pdb
total_actors = len(mapping_entity_int_dict)
total_actions = len(mapping_entity_se_dict)

subevent_dict = df['sub_event_type'].value_counts().to_dict()

graphs = []

for df_dict in [grouped_dfs[key] for key in sorted(grouped_dfs.keys())]:

    node_data = {
        "actor": total_actors,
        "action": total_actions
    }
    # node_data = {
    #     "actor": total_actors,
    
    # }
    
    # node_data.update(subevent_dict)

    src_nodes, dst_nodes = get_src_dst_tensors(get_edges_unidirectional(df_dict,'actor1_i','actor2_i'))
    src_nodes_action, dst_nodes_action = get_src_dst_tensors(get_edges_unidirectional(df_dict,'actor1_i','sub_event_type_i'))

    edge_data = {
        ("actor", "involved_with", "actor"): (src_nodes, dst_nodes),
        ("actor", "involved_in", "action"): (src_nodes_action, dst_nodes_action)
    }

    # edges = {}
    # for e in k_subevent:
    #     edges[("actor","involved_in_action", e)] = get_src_dst_tensors(get_edges_action(grouped_dfs[1997], e))
    # edge_data.update(edges)
    # print(edges)

    graph = dgl.heterograph(edge_data, num_nodes_dict=node_data)
    graph.nodes["actor"].data["_ID"] = torch.arange(total_actors)
    graph.nodes["action"].data["_ID"] = torch.arange(total_actions)
    # for ntype in graph.ntypes:
    #     graph.nodes[ntype].data["_ID"] = torch.arange(node_data[ntype])
    # graph.nodes["actor2"].data["_ID"] = torch.arange(total_actors)
    # graph.nodes["action"].data["features"] = torch.arange(total_actions)
    graphs.append(graph)


In [56]:
graphs[0].ndata

defaultdict(<class 'dict'>, {'_ID': {'action': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24]), 'actor': tensor([    0,     1,     2,  ..., 18619, 18620, 18621])}})

In [53]:
len(graphs)

28

In [57]:
dgl.save_graphs("graph_data_acled.bin", graphs)

In [ ]:
# # Define node types and counts
# node_data = {
#     "actor": len(mapping_entity_int_dict),
#     "action": len(mapping_entity_se_dict),
#     "institution": 8740,
#     "field_of_study": 59965,
# }

# # Define edge types and their connections
# edge_data = {
#     ("author", "affiliated_with", "institution"): ([], []),
#     ("author", "writes", "paper"): ([], []),
#     ("paper", "cites", "paper"): ([], []),
#     ("paper", "has_topic", "field_of_study"): ([], []),
# }

# # Example: Populate edge data with dummy values (replace with real data)
# edge_data[("author", "affiliated_with", "institution")] = (
#     torch.randint(0, node_data["author"], (1000,)),
#     torch.randint(0, node_data["institution"], (1000,)),
# )
# edge_data[("author", "writes", "paper")] = (
#     torch.randint(0, node_data["author"], (10000,)),
#     torch.randint(0, node_data["paper"], (10000,)),
# )
# edge_data[("paper", "cites", "paper")] = (
#     torch.randint(0, node_data["paper"], (20000,)),
#     torch.randint(0, node_data["paper"], (20000,)),
# )
# edge_data[("paper", "has_topic", "field_of_study")] = (
#     torch.randint(0, node_data["paper"], (5000,)),
#     torch.randint(0, node_data["field_of_study"], (5000,)),
# )

# # Create the heterogeneous graph
# graph = dgl.heterograph(edge_data, num_nodes_dict=node_data)

# # Add features to the paper nodes (word2vec features)
# graph.nodes["paper"].data["features"] = torch.randn(node_data["paper"], 128)

# # Add labels for the prediction task (venues, 349 classes)
# graph.nodes["paper"].data["labels"] = torch.randint(0, 349, (node_data["paper"],))

# # Example: Split data based on publication year (dummy split)
# train_mask = torch.zeros(node_data["paper"], dtype=torch.bool)
# val_mask = torch.zeros(node_data["paper"], dtype=torch.bool)
# test_mask = torch.zeros(node_data["paper"], dtype=torch.bool)

# # Simulate paper publication years (replace with actual years from the dataset)
# publication_years = torch.randint(2015, 2020, (node_data["paper"],))

# # Define masks
# train_mask[publication_years < 2018] = True
# val_mask[publication_years == 2018] = True
# test_mask[publication_years > 2018] = True

# graph.nodes["paper"].data["train_mask"] = train_mask
# graph.nodes["paper"].data["val_mask"] = val_mask
# graph.nodes["paper"].data["test_mask"] = test_mask

# # Save the graph
# output_path = "ogbn_mag_demo.dgl"
# dgl.save_graphs(output_path, [graph])

# print(f"Graph saved to {output_path}")

Graph saved to ogbn_mag_demo.dgl


In [ ]:
glist, label_dict = load_graphs('graph_data_acled.dgl')

In [ ]:
glist

[Graph(num_nodes={'action': 15, 'actor': 137},
       num_edges={('actor', 'involved_in', 'action'): 650},
       metagraph=[('actor', 'action', 'involved_in')])]

In [ ]:
glist, label_dict = load_graphs('ogbn_mag_demo.dgl')